In [18]:
import plotly.express as px
import numpy as np
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

# Data Prep

In [61]:
df = pd.read_csv('./Data/world_info.csv')

path = './Data/Countries'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

countries = []
for c in onlyfiles:
    df = pd.read_csv(path+'/'+c)
    df['Country'] = str.split(c,'.')[0]
    df['Dates'] = df['Dates'].astype(str) + ' 2020'
    df['Dates'] = pd.to_datetime(df['Dates'],format='%b %d %Y')
    df.sort_values(by=['Dates'],inplace=True)
    df.reset_index(inplace=True)
    countries.append(df)
    
df = pd.concat(countries)
df = df[['Country','Dates','Cases','Deaths','Active Cases','Daily Deaths']]

dt = pd.DataFrame(np.random.randn(len(df['Dates'].unique()), 1), index=df['Dates'].unique())
dt.sort_index(inplace=True)
dt.reset_index(inplace=True)
dt = dt[['index']]
dt.reset_index(inplace=True)
dt.columns = ['Day0','Dates']

df = df.merge(dt,how='left',left_on='Dates',right_on='Dates',copy=False)
#df = df.sort_values(by='Day0')
df.reset_index(inplace=True,drop=True)

#Adding an extra row called 'sort' and bringing all 'China' data to the top to ensure the animation works properly
df["sort"] = df['Day0']                                                
for index, row in df.iterrows():
    if row['Country'] == 'China':
        df.loc[index,'sort'] = 0

df = df.sort_values(by='sort')        

df.to_csv('./Data/Test/timeseries.csv')
df


,Country,Dates,Cases,Deaths,Active Cases,Daily Deaths,Day0,sort
270,China,2020-02-21,76288,2345,53284,109,30,0
275,China,2020-02-26,78497,2744,43258,29,35,0
276,China,2020-02-27,78824,2788,39919,44,36,0
277,China,2020-02-28,79251,2835,37414,47,37,0
278,China,2020-02-29,79824,2870,35129,35,38,0
...,...,...,...,...,...,...,...,...
543,India,2020-03-25,657,12,602,2,63,63
1063,S,2020-03-25,9137,126,5281,6,63,63
39,Algeria,2020-03-25,302,21,216,2,63,63
423,Germany,2020-03-25,37323,206,33570,47,63,63


# Area Chart

In [ ]:
for col in ['Cases', 'Deaths','Active Cases','Daily Deaths']:
    
    fig = px.area(df, x="Dates", y=col, color="Country",)
    fig.update_layout(title=col+' by Country over Time')
    fig.show()

In [62]:
fig = px.scatter(df, x="Cases", y="Deaths", animation_frame="Day0",
           size="Daily Deaths", color="Country", hover_name="Country",animation_group="Country",text="Country",
                 range_x=[0,100000], range_y=[0,10000],trendline='ols')
fig.show()